In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
        
from torch.autograd import Variable
        
! pip install tensorly
import tensorly as tl
from torch.autograd import Variable
tl.set_backend('pytorch')

In [ ]:
class VQC(nn.Module):
  def __init__(self, n, num_layers):
    super(VQC, self).__init__()
    self.n = n
    self.num_layers = num_layers
    self.params = nn.parameter.Parameter(torch.randn(self.n, 3*self.num_layers))

  def prints(self,psi):
    print("-------------------------MPS STRUCTURE SHAPE------------------------------------------")
    for _,i in enumerate(psi):
      print(i.shape)
    print("-------------------------SHAPE------------------------------------------")

  def H(self, i, psi):
    H_matrix=1/((2)**0.5)*torch.tensor([[1, 1],
                                [1,-1]],dtype=torch.cfloat)
    psi[i]=torch.tensordot(H_matrix,psi[i],([1],[2])) 
    psi[i]=torch.moveaxis(psi[i],0,2)
    return psi
  def RX(self,i,j,psi): # Alter the array psi into the array where H has acted on the ith qubit
    # RX=torch.tensor([[torch.cos(i/2), torch.sin(i/2)*-1j],
    #                         [torch.sin(i/2)*-1j,torch.cos(i/2)]],dtype=torch.cfloat)

    t1=(torch.cos(i/2)).reshape(1)
    t2=(torch.sin(i/2)*-1j).reshape(1)
    t3=(torch.sin(i/2)*-1j).reshape(1)
    t4=(torch.cos(i/2)).reshape(1)
    f1=torch.cat([t1,t2])
    f2=torch.cat([t3,t4])
    rx=torch.cat([f1,f2]).reshape(2,2).type(torch.cfloat)
    psi[j]=torch.tensordot(rx,psi[j],([1],[2])) 
    psi[j]=torch.moveaxis(psi[j],0,2)
    return psi

  def RY(self,i,j,psi): # Alter the array psi into the array where H has acted on the ith qubit
    t1=(torch.cos(i/2)).reshape(1)
    t2=-1.*(torch.sin(i/2)).reshape(1)
    t3=(torch.sin(i/2)).reshape(1)
    t4=(torch.cos(i/2)).reshape(1)
    f1=torch.cat([t1,t2])
    f2=torch.cat([t3,t4])
    ry=torch.cat([f1,f2]).reshape(2,2).type(torch.cfloat)
    psi[j]=torch.tensordot(ry,psi[j],([1],[2])) 
    psi[j]=torch.moveaxis(psi[j],0,2)
    return psi

  def RZ(self,i,j,psi): # Alter the array psi into the array where H has acted on the ith qubit
    t1=(torch.exp(-1j*i/2)).reshape(1)
    t2=(torch.zeros(1)).reshape(1)
    t3=(torch.zeros(1)).reshape(1)
    t4=(torch.exp(1j*i/2)).reshape(1)
    f1=torch.cat([t1,t2])
    f2=torch.cat([t3,t4])
    rz=torch.cat([f1,f2]).reshape(2,2).type(torch.cfloat)
    psi[j]=torch.tensordot(rz,psi[j],([1],[2])) 
    psi[j]=torch.moveaxis(psi[j],0,2)
    return psi

  def CNOT(self, i, j, psi):
     dim = psi[i].shape[0]
     mps=torch.tensordot(psi[i],psi[j],([1],[0]))
     mps=torch.moveaxis(mps,2,1)
     CNOT_matrix=torch.tensor([[1,0,0,0],
                       [0,1,0,0],
                       [0,0,0,1],
                       [0,0,1,0]],dtype=torch.cfloat)
     CNOT_tensor=torch.reshape(CNOT_matrix, (2,2,2,2))
     mps=torch.tensordot(CNOT_tensor, mps, ([2,3],[2, 3]))
     mps=torch.moveaxis(mps,1,3).reshape((2*dim,2*dim))
     #mps+=mps+(torch.randn(mps.shape))*0.001
     data=torch.rand(mps.shape)
     data=data*mps.abs().max()*0.000001/data.max()
     mps=mps+data
     #u,s,v = torch.linalg.svd(mps)
     u,s,v = torch.svd(mps)
     v = torch.conj(v.T)
     c = torch.diag(s[:dim])
     mps1 = torch.mm(u[:,:dim], c.type(torch.cfloat))
     mps1 = mps1.reshape((2,dim, dim))
     psi[i] = torch.moveaxis(mps1,0,2)
     mps2 = v[:dim,:]
     psi[j] = mps2.reshape((dim, dim,2))
     return psi

  def CNOT_full(self,i,j,psi):
        # Alter the array psi into the array where H has acted on the ith qubit
        #i-->control;
        #j-->target;
        s0=psi[i].shape[0]
        s1=psi[j].shape[1]
        # print(i,j)
        # print(psi[i].shape)
        # print(psi[j].shape)
        mps=torch.tensordot(psi[i],psi[j],([1],[0]))
        mps=torch.moveaxis(mps,2,1)
        #print(mps.shape)
        CNOT_matrix=torch.tensor([[1,0,0,0],
                      [0,1,0,0],
                      [0,0,0,1],
                      [0,0,1,0]],dtype=torch.cfloat)

        CNOT_tensor=torch.reshape(CNOT_matrix, (2,2,2,2))
        mps=torch.tensordot(CNOT_tensor, mps, ([2,3],[2, 3]))
        mps=torch.moveaxis(mps,1,3).reshape((s0*2,s1*2)) 
        data=torch.rand(mps.shape)
        data=data*mps.abs().max()*0.00001/data.max()
        mps=mps+data
        u, s, v =  torch.linalg.svd(mps)
        c=(torch.zeros(u.shape[1],v.shape[0]))
        for h,k in enumerate(s):
          c[h,h]=k
        mps1=torch.mm(u, c.type(torch.cfloat))
        mps1=mps1.reshape((2,int(mps1.shape[0]/2),mps1.shape[1]))
        psi[i]=torch.moveaxis(mps1,0,2)
        mps2=v#.transpose(-2, -1)
        psi[j]=mps2.reshape((mps2.shape[0],int(mps2.shape[1]/2),2))   
        return psi

  def getParams(self):
    return self.params
  
  def setParams(self, params_matrix):
    self.params = params_matrix

  def compute_tensor_circuit(self, psi, t):
    #Embedding layer
    for i in range(self.n):
      self.RX(t[i], i, psi)
      self.H(i, psi)

    # Parametrized Layers
    #Each layer contains a series of parametrized RX, RY, RZ gates and cyclic entanglement
    for i in range(self.num_layers):
      #CNOTs
      for j in range(self.n-1):
        psi = self.CNOT(j, j+1, psi)
      psi = self.CNOT(self.n-1,0, psi)

      #Parametrized rotations
      for j in range(self.n):
        psi =self.RX(self.params[j][3*i + 0], j, psi)
        psi = self.RY(self.params[j][3*i + 1], j, psi)
        psi = self.RZ(self.params[j][3*i + 2], j, psi)

    return psi

  def contract_tensor_ring(self,psi):
    psi_new = psi[0]
    for i in range(self.n-2):
      psi_new = torch.tensordot(psi_new, psi[i+1], ([1], [0]))
      psi_new = torch.moveaxis(psi_new, -2, 1)

    psi_new = torch.tensordot(psi_new, psi[self.n-1], ([0,1], [1,0]))
    return psi_new

  def forward(self, psi, t):

    psi_final = self.compute_tensor_circuit(psi, t)
    k = self.contract_tensor_ring(psi_final)
    k1 = k[0, 0, 0, 0].abs().reshape(1) ** 2
    k2 = k[0, 0, 1, 0].abs().reshape(1) ** 2
    k3 = k[1, 1, 1, 1].abs().reshape(1) ** 2

    return torch.cat([k1, k2, k3]).reshape(1, 3)

  def final_state(self, psi,t):

    #psi_final = self.test_circuit(psi,t)
    psi_final = self.compute_tensor_circuit(psi, t)
    k = self.contract_tensor_ring(psi_final)
    return k


In [ ]:
def create_zero_tensor(num_qubits, d):
  psi_0 = []
  for j in range(num_qubits):
    k = torch.zeros((d,d,2), dtype=torch.cfloat)
    k[0,0,0] = 1.0
    psi_0.append(k)
  return psi_0

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
plt.style.use('ggplot')

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler

iris = load_iris()
X = iris['data']
y = iris['target']
#X=X[y!=2]
#y=y[y!=2]
names = iris['target_names']
feature_names = iris['feature_names']

# Scale data to have mean 0 and variance 1 
# which is importance for convergence of the neural network
#scaler = StandardScaler()
scaler = MinMaxScaler((-3.142, 3.142))
X_scaled = scaler.fit_transform(X)

# Split the data set into training and testing
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(
    X_scaled, y, test_size=0.25, random_state=2)

In [ ]:
import tqdm
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
num_qubits = 4
num_layers = 1
bond = 16

In [ ]:
model_test = VQC(num_qubits,num_layers)
optimizer = torch.optim.Adam(model_test.parameters(), lr = 0.01, weight_decay=0.001)
loss_fn = nn.CrossEntropyLoss()
EPOCHS = 25
X_train = (torch.from_numpy(X_train_0)).float()
y_train = (torch.from_numpy(y_train_0)).long().reshape(X_train.shape[0],1)
X_test  = Variable(torch.from_numpy(X_test_0)).float()
y_test  = Variable(torch.from_numpy(y_test_0)).long().reshape(X_test.shape[0],1)
model_test.train()

loss_list     = np.zeros((EPOCHS,))
accuracy_list = np.zeros((EPOCHS,))
accuracy_list_train = np.zeros((EPOCHS,))

for epoch in tqdm.trange(EPOCHS):
  l = 0
  c = []
  for b,s in enumerate(X_train):
    psi = create_zero_tensor(num_qubits, bond)
    k1 = F.softmax(model_test(psi,s), dim = 1)
    loss = F.cross_entropy(k1, y_train[b])
    correct = (torch.argmax(k1, dim=1) == y_train[b]).type(torch.FloatTensor)
    c.append(correct.item())
    l+=loss/4
    if int((b+1)%4)==0:
      optimizer.zero_grad()
      l.backward(create_graph = True)
      optimizer.step()
      loss_list[epoch] = l.item()
      loss_epoch = float(l)
      l=torch.zeros(1)
    accuracy_list_train[epoch] = np.array(c).mean()
  with torch.no_grad():
    c=[]
    for j,s in enumerate(X_test):
      psi = create_zero_tensor(num_qubits, bond)
      k1 = F.softmax(model_test(psi,s), dim = 1)
      correct = (torch.argmax(k1, dim=1) == y_test[j]).type(torch.FloatTensor)
      c.append(correct.item())
    accuracy_list[epoch] = np.array(c).mean()
  print("Train Loss: {0:1.4f}; Train Accuracy: {1:1.4f}; Test Accuracy: {2:1.4f}".format(loss_epoch, accuracy_list_train[epoch], accuracy_list[epoch]))
